In [ ]:
#| default_exp bbox.core

# BBox Core

> API details.

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastcore.all import *
import math
import numpy as np
import polvo as pv

/home/lgvaz/git/polvo/polvo/utils/logging.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
#TODO: With points sorted, xmax,ymin, etc can be directly indexed

In [ ]:
np.reshape

<function numpy.reshape(a, newshape, order='C')>

In [ ]:
#|export
class _BBox:
    def __init__(self, points):
        self.points, _ = pv.sort_quadrilateral(np.array(points))

    @classmethod
    def from_points(cls, points): return cls(points)
    @classmethod
    def from_flat(cls, points): return cls(np.reshape(points, (4, 2)))

In [ ]:
#|export
class BBox(_BBox):
    """Bounding Box representation.
    Should **not** be instantiated directly, instead use `from_*` methods. e.g. `from_xyxy`, `from_xywh`.
    Is able to transform coordinates into different formats, e.g. `xyxy`, `xywh`.
    
    # Examples
    Create from `xywh` format, and get `xyxy` coordinates.
    ```python
    bbox = BBox.from_xywh(1, 1, 4, 4)
    xyxy = bbox.xyxy
    ```
    """
    def __repr__(self):
        return f"<{self.__class__.__name__} (xmin:{self.xmin}, ymin:{self.ymin}, xmax:{self.xmax}, ymax:{self.ymax})>"

    def __eq__(self, other) -> bool:
        if isinstance(other, BBox): return self.xyxyxyxy == other.xyxyxyxy
        return False
    
    @property
    def xmin(self): return self.points[:, 0].min()
    @property
    def xmax(self): return self.points[:, 0].max()
    @property
    def ymin(self): return self.points[:, 1].min()
    @property
    def ymax(self): return self.points[:, 1].max()

    @property
    def width(self): return self.xmax-self.xmin
    @property
    def height(self): return self.ymax-self.ymin
    @property
    def area(self): return self.width*self.height
    @property
    def xyxy(self): return self.xmin, self.ymin, self.xmax, self.ymax
    @property
    def yxyx(self): return self.ymin, self.xmin, self.ymax, self.xmax
    @property
    def xywh(self): return self.xmin, self.ymin, self.width, self.height
    @property
    def xyxyxyxy(self): return self.xmin,self.ymin, self.xmax,self.ymin, self.xmax,self.ymax, self.xmin,self.ymax

    def relative_xcycwh(self, img_width: int, img_height: int):
        scale = np.array([img_width, img_height, img_width, img_height])
        x, y, w, h = self.xywh / scale
        xc = x + 0.5*w
        yc = y + 0.5*h
        return (xc, yc, w, h)

    @classmethod
    def from_xywh(cls, x, y, w, h): return cls.from_xyxy(x, y, x+w, y+h)
    @classmethod
    def from_xyxy(cls, xmin, ymin, xmax, ymax): return cls.from_flat((xmin,ymin, xmax,ymin, xmax,ymax, xmin,ymax))
    @classmethod
    def from_relative_xcycwh(cls, xc, yc, bw, bh, img_width, img_height):
        # subtracting 0.5 goes from center to left/upper edge, adding goes to right/bottom
        pnts = [(xc - 0.5*bw), (yc - 0.5*bh), (xc + 0.5*bw), (yc + 0.5*bh)]
        # convert from relative to absolute coordinates
        scale = np.array([img_width, img_height, img_width, img_height])
        xmin, ymin, xmax, ymax = np.around(pnts * scale).astype(int).tolist()
        return cls.from_xyxy(xmin, ymin, xmax, ymax)
    @classmethod
    def from_rle(cls, rle, h, w):
        a = np.array(rle.counts, dtype=int)
        a = a.reshape((-1, 2))  # an array of (start, length) pairs
        a[:, 0] -= 1  # `start` is 1-indexed
        y0 = a[:, 0] % h
        y1 = y0 + a[:, 1]
        if np.any(y1 > h):
            # got `y` overrun, meaning that there are a pixels in mask on 0 and shape[0] position
            y0 = 0
            y1 = h
        else:
            y0 = np.min(y0)
            y1 = np.max(y1)
        x0 = a[:, 0] // h
        x1 = (a[:, 0] + a[:, 1]) // h
        x0 = np.min(x0)
        x1 = np.max(x1)
        if x1 > w:
            # just went out of the image dimensions
            raise ValueError(f"invalid RLE or image dimensions: x1={x1} > shape[1]={w}")
        return cls.from_xyxy(x0, y0, x1, y1)

In [ ]:
bbox = BBox.from_xyxy(1, 2, 3, 4)

test_eq(bbox.xyxy, (1, 2, 3, 4))
test_eq(bbox.yxyx, (2, 1, 4, 3))
test_eq(bbox.xywh, (1, 2, 2, 2))

In [ ]:
bbox = BBox.from_xywh(10, 72, 478, 72)
test_eq(bbox.xyxy, (10, 72, 10+478, 72+72))

In [ ]:
w, h = 640, 480
xcycwh = (0.7, 0.2, 0.1, 0.2)
bbox = BBox.from_relative_xcycwh(*xcycwh, img_width=w, img_height=h)

test_eq(bbox.xyxy, (416, 48, 480, 144))
test_close(bbox.relative_xcycwh(img_width=w, img_height=h), xcycwh)

In [ ]:
#|export
class OBBox(_BBox):
    @classmethod
    def from_clockwise(cls, x,y, w,h, degrees):
        r = math.radians(degrees)
        sinr, cosr = math.sin(r), math.cos(r)
        x2 = x+cosr*w
        y2 = y+sinr*w
        x3 = x+cosr*w-sinr*h
        y3 = y+sinr*w+cosr*h
        x4 = x-sinr*h
        y4 = y+cosr*h
        
        return cls.from_flat((x,y, x2,y2, x3,y3, x4,y4))
    
    def __repr__(self):
#         return f"<{self.__class__.__name__} (x1:{self.x1},y1:{self.y1}, x2:{self.x2},y2:{self.y2}, x3:{self.x3},y3:{self.y3}, x4:{self.x4},y4:{self.y4})>"
#         return f"<{self.__class__.__name__} (({self.x1},{self.y1}), ({self.x2},{self.y2}), ({self.x3},{self.y3}), ({self.x4},{self.y4}))>"
        return f"<{self.__class__.__name__} ({self.points})>"

In [ ]:
obbox1 = OBBox.from_flat((60,20, 60,120, 10,120, 10,20)) # non-rotated
test_eq(obbox1.points, np.array(((10,20), (60,20), (60,120), (10, 120))))

In [ ]:
obbox2 = OBBox.from_clockwise(60,20, 100,50, math.degrees(math.asin(4/5))) #rotated 
obbox3 = OBBox.from_flat((60,20, 120,100, 80,130, 20,50)) # rotated 
test_close(obbox2.points, obbox3.points)

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()